In [673]:
import sys
import pandas as pd
import numpy as np
sys.path.append("..") # 시스템 경로 변경(MySQLConnect 클래스 부르기 위함)

In [674]:
from MySQLConnect import MySQLConnect

In [675]:
# 접속정보
host = 'kmaster.czypjmusens3.ap-northeast-2.rds.amazonaws.com'
user = 'KMaster'
password = 'kookmin1234'
db = 'km_db'

In [676]:
dc = MySQLConnect(host, user, password, db)

del


In [677]:
# 조회조건 들어갈 변수
block_nm = 'A17';A_6 = '59형'; A_11 = '본인'; general_1 = '일%'; general_2 = 'N%'; general_3 = '';

In [678]:
# 데이터 담을 dataframe
full_data = pd.DataFrame(index = range(1))

In [679]:
# 일반공급 Query 시에만 사용
A11_general = pd.DataFrame(list(dc.getRecord("*", "LEASE_DATA", "BLOCK_NM = '"+block_nm+"' and A_6 = '"+A_6+"' and A_11 = '"+A_11+"' and (W_14 like '"+general_1+"' or W_14 like '"+general_2+"' or W_14 = '"+general_3+ "') " )))

In [680]:
len(A11_general) # 전체인원수

449

In [681]:
full_data['신청인원'] = len(A11_general)

###### 일반공급 이면서 당첨/탈락 으로 분류

In [682]:
A11_general_win = A11_general.ix[A11_general.W_4=="당첨"]
A11_general_lose = A11_general.ix[A11_general.W_4!="당첨"]

In [683]:
len(A11_general_win) # 당첨 : 57명

44

In [684]:
len(A11_general_lose) # 탈락 : 63명

405

###### 일반공급 ###########################################################################################################
###### 소득70%적용(W_16 == 'Y') -> 당첨자신청순위(W_13 : 3순위부터) -> 3자녀이상여부(A_33) (Y/N 순으로)
###### 소득70%적용(W_16 != 'Y') -> 당첨자신청순위(W_13 : 3순위부터) -> 3자녀이상여부(A_33) (Y/N 순으로)
###### 일반공급 ###########################################################################################################

In [685]:
A11_general_win.groupby(['W_16', 'W_13', 'A_33'])['W_21'].min() # A_33 : 3자녀이상 여부

W_16  W_13  A_33
      1순위   N       10
            Y        4
Y     1순위   N       10
            Y        6
Name: W_21, dtype: int64

In [736]:
full_data['커트라인점수'] = str(6)+'(70%/1순위/Y)' # 3자녀 여부만 확인

In [737]:
full_data

,신청인원,커트라인점수,50%이하_당첨,50%이하_탈락,50%초과_당첨,50%초과_탈락,세대주_50세이상_당첨,세대주_50세이상_탈락,세대주_40세이상_당첨,세대주_40세이상_탈락,...,65세_미부양_당첨,65세_미부양_탈락,제조업_근무_당첨,제조업_근무_탈락,제조업_미근무_당첨,제조업_미근무_탈락,사회취약계층_해당_당첨,사회취약계층_해당_탈락,사회취약계층_미해당_당첨,사회취약계층_미해당_탈락
0,449,6(70%/1순위/Y),29,300,15,105,17,140,15,92,...,35,370,2,18,42,387,4,20,40,385


###### 50% 이하 / 초과 데이터 나누기(A11_general : 원본 데이터 에서 진행)

In [688]:
## 일반 당첨 50%이하/초과
A11_general_win_50down = A11_general_win.ix[A11_general_win.A_32 == '50%이하']
A11_general_win_50up = A11_general_win.ix[A11_general_win.A_32 != '50%이하']
## 일반 탈락 50%이하/초과
A11_general_lose_50down = A11_general_lose.ix[A11_general_lose.A_32 == '50%이하']
A11_general_lose_50up = A11_general_lose.ix[A11_general_lose.A_32 != '50%이하']

In [689]:
full_data['50%이하_당첨'] = len(A11_general_win_50down)

In [690]:
full_data['50%이하_탈락'] = len(A11_general_lose_50down)

In [691]:
full_data['50%초과_당첨'] = len(A11_general_win_50up)

In [692]:
full_data['50%초과_탈락'] = len(A11_general_lose_50up)

##### 세대주 나이 구간화

In [693]:
bins = [29,39,49,99] # range 범위 지정
group_names = ['30대이상', '40대이상', '50대이상'] # 지정할 그룹명
categories_win = pd.cut(A11_general_win['A_34'], bins, labels=group_names) # 일반 당첨
categories_lose = pd.cut(A11_general_lose['A_34'], bins, labels=group_names) # 일반 당첨

In [694]:
A11_general_win['세대주나이분류'] = categories_win
A11_general_lose['세대주나이분류'] = categories_lose

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [695]:
## 일반 당첨 30/40/50
A11_general_win_30 = A11_general_win.ix[A11_general_win.세대주나이분류 == '30대이상']
A11_general_win_40 = A11_general_win.ix[A11_general_win.세대주나이분류 == '40대이상']
A11_general_win_50 = A11_general_win.ix[A11_general_win.세대주나이분류 == '50대이상']
## 일반 탈락 30/40/50
A11_general_lose_30 = A11_general_lose.ix[A11_general_lose.세대주나이분류 == '30대이상']
A11_general_lose_40 = A11_general_lose.ix[A11_general_lose.세대주나이분류 == '40대이상']
A11_general_lose_50 = A11_general_lose.ix[A11_general_lose.세대주나이분류 == '50대이상']

In [696]:
full_data['세대주_50세이상_당첨'] = len(A11_general_win_50)

In [697]:
full_data['세대주_50세이상_탈락'] = len(A11_general_lose_50)

In [698]:
full_data['세대주_40세이상_당첨'] = len(A11_general_win_40)

In [699]:
full_data['세대주_40세이상_탈락'] = len(A11_general_lose_40)

In [700]:
full_data['세대주_30세이상_당첨'] = len(A11_general_win_30)

In [701]:
full_data['세대주_30세이상_탈락'] = len(A11_general_lose_30)

##### 부양가족 수 구간화

In [702]:
child_bins = [0,1,2,9] # range 범위 지정
child_group_names = ['1인', '2인', '3인'] # 지정할 그룹명
child_categories_win = pd.cut(A11_general_win['A_35'], child_bins, labels=child_group_names) # 일반 부양 당첨
child_categories_lose = pd.cut(A11_general_lose['A_35'], child_bins, labels=child_group_names) # 일반 부양 탈락

In [703]:
A11_general_win['부양가족구분'] = child_categories_win
A11_general_lose['부양가족구분'] = child_categories_lose

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [704]:
# 일반 당첨 1/2/3인
A11_general_win_child1 = A11_general_win.ix[A11_general_win['부양가족구분'] == '1인']
A11_general_win_child2 = A11_general_win.ix[A11_general_win['부양가족구분'] == '2인']
A11_general_win_child3 = A11_general_win.ix[A11_general_win['부양가족구분'] == '3인']
# 일반 탈락 1/2/3인
A11_general_lose_child1 = A11_general_lose.ix[A11_general_lose['부양가족구분'] == '1인']
A11_general_lose_child2 = A11_general_lose.ix[A11_general_lose['부양가족구분'] == '2인']
A11_general_lose_child3 = A11_general_lose.ix[A11_general_lose['부양가족구분'] == '3인']

In [705]:
full_data['부양가족_3인_당첨'] = len(A11_general_win_child3)
full_data['부양가족_3인_탈락'] = len(A11_general_lose_child3)
full_data['부양가족_2인_당첨'] = len(A11_general_win_child2)
full_data['부양가족_2인_탈락'] = len(A11_general_lose_child2)
full_data['부양가족_1인_당첨'] = len(A11_general_win_child1)
full_data['부양가족_1인_탈락'] = len(A11_general_lose_child1)

##### 거주기간 구간화

In [706]:
resi_bins = [1,3,5,100] # range 범위 지정
resi_group_names = ['1_3년', '3_5년', '5년이상'] # 지정할 그룹명
resi_categories_win = pd.cut(A11_general_win['A_22'],  resi_bins,False, labels=resi_group_names) # 일반 거주 당첨
resi_categories_lose = pd.cut(A11_general_lose['A_22'],  resi_bins,False, labels=resi_group_names) # 일반 거주 탈락

In [707]:
A11_general_win['거주기간구분'] = resi_categories_win
A11_general_lose['거주기간구분'] = resi_categories_lose

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [708]:
# 일반 당첨 
A11_general_win_oneTothree = A11_general_win.ix[A11_general_win.거주기간구분 == "1_3년"]
A11_general_win_threeTofive = A11_general_win.ix[A11_general_win.거주기간구분 == "3_5년"]
A11_general_win_five = A11_general_win.ix[A11_general_win.거주기간구분 == "5년이상"]
# 일반 탈락
A11_general_lose_oneTothree = A11_general_lose.ix[A11_general_lose.거주기간구분 == "1_3년"]
A11_general_lose_threeTofive = A11_general_lose.ix[A11_general_lose.거주기간구분 == "3_5년"]
A11_general_lose_five = A11_general_lose.ix[A11_general_lose.거주기간구분 == "5년이상"]

###### 1~3년 이상 당첨/탈락 데이터 나누기

In [709]:
full_data['거주기간_5년_당첨'] = len(A11_general_win_five)
full_data['거주기간_5년_탈락'] = len(A11_general_lose_five)
full_data['거주기간_3_5년_당첨'] = len(A11_general_win_threeTofive)
full_data['거주기간_3_5년_탈락'] = len(A11_general_lose_threeTofive)
full_data['거주기간_1_3년_당첨'] = len(A11_general_win_oneTothree)
full_data['거주기간_1_3년_탈락'] = len(A11_general_lose_oneTothree)

###### 미성년자 3명이상 여부 데이터 나누기

In [710]:
# 일반 당첨 
A11_general_win_threeup = A11_general_win.ix[A11_general_win.W_17 >= 3]
A11_general_win_threedown = A11_general_win.ix[A11_general_win.W_17 < 3 ]
# 일반 탈락
A11_general_lose_threeup = A11_general_lose.ix[A11_general_lose.W_17 >= 3]
A11_general_lose_threedown = A11_general_lose.ix[A11_general_lose.W_17 < 3 ]

In [711]:
full_data['미성년_3명이상_당첨'] = len(A11_general_win_threeup)

In [712]:
full_data['미성년_3명이상_탈락'] = len(A11_general_lose_threeup)

In [713]:
full_data['미성년_3명미만_당첨'] = len(A11_general_win_threedown)

In [714]:
full_data['미성년_3명미만_탈락'] = len(A11_general_lose_threedown)

In [715]:
## 일반 당첨
A11_general_win_structY = A11_general_win.ix[A11_general_win.W_36 == 'Y'] 
A11_general_win_structN = A11_general_win.ix[A11_general_win.W_36 != 'Y']
## 일반 탈락
A11_general_lose_structY = A11_general_lose.ix[A11_general_lose.W_36 == 'Y'] 
A11_general_lose_structN = A11_general_lose.ix[A11_general_lose.W_36 != 'Y']

In [716]:
full_data['건설근로자_3점_당첨'] = len(A11_general_win_structY)

In [717]:
full_data['건설근로자_3점_탈락'] = len(A11_general_lose_structY)

In [718]:
full_data['건설근로자_0점_당첨'] = len(A11_general_win_structN)

In [719]:
full_data['건설근로자_0점_탈락'] = len(A11_general_lose_structN) 

##### 청약 불입회수 분리

In [720]:
not_save_bins = [36,48,60,200] # range 범위 지정
not_save_group_names = ['36회이상', '48회이상', '60회이상'] # 지정할 그룹명
not_save_categories_win = pd.cut(A11_general_win['A_46'],  not_save_bins,False, labels=not_save_group_names) # 일반당첨 분류
not_save_categories_lose = pd.cut(A11_general_lose['A_46'],  not_save_bins,False, labels=not_save_group_names) # 일반탈락 분류

In [721]:
A11_general_win['청약횟수분리'] = not_save_categories_win
A11_general_lose['청약횟수분리'] = not_save_categories_lose

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [722]:
# 일반 당첨
A11_general_win_36 = A11_general_win.ix[A11_general_win.청약횟수분리 == '36회이상']
A11_general_win_48 = A11_general_win.ix[A11_general_win.청약횟수분리 == '48회이상']
A11_general_win_60 = A11_general_win.ix[A11_general_win.청약횟수분리 == '60회이상']
# 일반 탈락
A11_general_lose_36 = A11_general_lose.ix[A11_general_lose.청약횟수분리 == '36회이상']
A11_general_lose_48 = A11_general_lose.ix[A11_general_lose.청약횟수분리 == '48회이상']
A11_general_lose_60 = A11_general_lose.ix[A11_general_lose.청약횟수분리 == '60회이상']

In [723]:
full_data['청약저축_60이상_당첨'] = len(A11_general_win_60)
full_data['청약저축_60이상_탈락'] = len(A11_general_lose_60)
full_data['청약저축_48이상_당첨'] = len(A11_general_win_48)
full_data['청약저축_48이상_탈락'] = len(A11_general_lose_48)
full_data['청약저축_36이상_당첨'] = len(A11_general_win_36)
full_data['청약저축_36이상_탈락'] = len(A11_general_lose_36)

###### 계약사실 분리 데이터

In [724]:
## 계약사실 분리 함수
def f(x) :
    if x['W_24'] == -5:
        return "1년이내"
    elif x['W_24'] == -3:
        return "3년이내" 
    else:
        return "해당없음"

In [725]:
A11_general_win['계약사실구분'] = A11_general_win.apply(f, axis=1)
A11_general_lose['계약사실구분'] = A11_general_lose.apply(f, axis=1)

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


###### 계약사실 데이터 1년이내/3년이내 나누기

In [726]:
## 일반 당첨
A11_general_win_one = A11_general_win.ix[A11_general_win.계약사실구분 == "1년이내"]
A11_general_win_three = A11_general_win.ix[A11_general_win.계약사실구분 == "3년이내"]
## 일반 탈락
A11_general_lose_one = A11_general_lose.ix[A11_general_lose.계약사실구분 == "1년이내"]
A11_general_lose_three = A11_general_lose.ix[A11_general_lose.계약사실구분 == "3년이내"]

In [727]:
full_data['계약사실_1년이내_당첨'] = len(A11_general_win_one)

In [728]:
full_data['계약사실_1년이내_탈락'] = len(A11_general_lose_one)

In [729]:
full_data['게약사실_3년이내_당첨'] = len(A11_general_win_three)

In [730]:
full_data['게약사실_3년이내_탈락'] = len(A11_general_lose_three)

In [731]:
## 일반 당첨 
A11_general_win_parent_care = A11_general_win.ix[A11_general_win.W_30 =='Y']
A11_general_win_parent_careless = A11_general_win.ix[A11_general_win.W_30 != 'Y']
A11_general_win_manufac = A11_general_win.ix[A11_general_win.W_31 == 'Y']
A11_general_win_not_manufac = A11_general_win.ix[A11_general_win.W_31 != 'Y']
A11_general_win_weekness_o = A11_general_win.ix[A11_general_win.W_32 == 'Y']
A11_general_win_weekness_x = A11_general_win.ix[A11_general_win.W_32 != 'Y']
A11_general_win_dang_one = A11_general_win.ix[A11_general_win.W_13 == '1순위']
A11_general_win_dang_two = A11_general_win.ix[A11_general_win.W_13 == '2순위']
A11_general_win_dang_three = A11_general_win.ix[A11_general_win.W_13 == '3순위']
## 일반 탈락
A11_general_lose_parent_care = A11_general_lose.ix[A11_general_lose.W_30 =='Y']
A11_general_lose_parent_careless = A11_general_lose.ix[A11_general_lose.W_30 != 'Y']
A11_general_lose_manufac = A11_general_lose.ix[A11_general_lose.W_31 == 'Y']
A11_general_lose_not_manufac = A11_general_lose.ix[A11_general_lose.W_31 != 'Y']
A11_general_lose_weekness_o = A11_general_lose.ix[A11_general_lose.W_32 == 'Y']
A11_general_lose_weekness_x = A11_general_lose.ix[A11_general_lose.W_32 != 'Y']
A11_general_lose_dang_one = A11_general_lose.ix[A11_general_lose.W_13 == '1순위']
A11_general_lose_dang_two = A11_general_lose.ix[A11_general_lose.W_13 == '2순위']
A11_general_lose_dang_three = A11_general_lose.ix[A11_general_lose.W_13 == '3순위']

In [732]:
full_data['당해_1순위_당첨'] = len(A11_general_win_dang_one)
full_data['당해_1순위_탈락'] = len(A11_general_lose_dang_one)
full_data['당해_2순위_당첨'] = len(A11_general_win_dang_two)
full_data['당해_2순위_탈락'] = len(A11_general_lose_dang_two)
full_data['당해_3순위_당첨'] = len(A11_general_win_dang_three)
full_data['당해_3순위_탈락'] = len(A11_general_lose_dang_three)
full_data['65세_부양_당첨'] = len(A11_general_win_parent_care)
full_data['65세_부양_탈락'] = len(A11_general_lose_parent_care)
full_data['65세_미부양_당첨'] = len(A11_general_win_parent_careless)
full_data['65세_미부양_탈락'] = len(A11_general_lose_parent_careless)
full_data['제조업_근무_당첨'] = len(A11_general_win_manufac)
full_data['제조업_근무_탈락'] = len(A11_general_lose_manufac)
full_data['제조업_미근무_당첨'] = len(A11_general_win_not_manufac)
full_data['제조업_미근무_탈락'] = len(A11_general_lose_not_manufac)
full_data['사회취약계층_해당_당첨'] = len(A11_general_win_weekness_o)
full_data['사회취약계층_해당_탈락'] = len(A11_general_lose_weekness_o)
full_data['사회취약계층_미해당_당첨'] = len(A11_general_win_weekness_x)
full_data['사회취약계층_미해당_탈락'] = len(A11_general_lose_weekness_x)

In [738]:
full_data # 잘들어갔는지 확인

,신청인원,커트라인점수,50%이하_당첨,50%이하_탈락,50%초과_당첨,50%초과_탈락,세대주_50세이상_당첨,세대주_50세이상_탈락,세대주_40세이상_당첨,세대주_40세이상_탈락,...,65세_미부양_당첨,65세_미부양_탈락,제조업_근무_당첨,제조업_근무_탈락,제조업_미근무_당첨,제조업_미근무_탈락,사회취약계층_해당_당첨,사회취약계층_해당_탈락,사회취약계층_미해당_당첨,사회취약계층_미해당_탈락
0,449,6(70%/1순위/Y),29,300,15,105,17,140,15,92,...,35,370,2,18,42,387,4,20,40,385


In [739]:
## 엑셀저장block_nm = '';A_6 = ''; A_11 = '';A_24 = '';
filename = block_nm+"_"+A_6+"_"+"일반공급.xlsx"
writer = pd.ExcelWriter(filename)
full_data.to_excel(writer,'Sheet1')
writer.save()

In [735]:
dc.dbCursor.close()